In [ ]:
import azmeta.notebook.interactive as azmi
import pandas as pd
azmi.connect_kqlmagic()

# Parameters

In [ ]:
location = 'EastUS2'

In [ ]:
%%kql -ecbp
let last5bills = (Usage | distinct BillingPeriodStartDate | top 5 by BillingPeriodStartDate);
Usage
| where BillingPeriodStartDate in (last5bills)
| {azmeta_kql_usage_scope}
| where ConsumedService == 'Microsoft.Compute' and MeterCategory == 'Virtual Machines' and isempty(ReservationId)
| summarize sum(Cost) by BillingPeriodStartDate, MeterSubCategory
| evaluate pivot(MeterSubCategory, sum(sum_Cost))
| extend BillingPeriodStartDate = format_datetime(BillingPeriodStartDate, 'yyyy-MM')
| sort by BillingPeriodStartDate asc
| render columnchart with (ytitle="$", xtitle="Billing Period", title="Unreserved VM Usage by Series: Past 5 Bills")

In [ ]:
%%kql rs << -ecbp
let latestBill = toscalar(Usage | summarize max(BillingPeriodStartDate));
Usage
| where BillingPeriodStartDate == latestBill
| {azmeta_kql_usage_scope}
| where ConsumedService == 'Microsoft.Compute' and MeterCategory == 'Virtual Machines' and isempty(ReservationId)
| summarize sum(Cost) by ResourceLocation, MeterSubCategory
| evaluate pivot(ResourceLocation, sum(sum_Cost))

In [ ]:
df: pd.DataFrame = rs.to_dataframe().set_index('MeterSubCategory')
df.style.background_gradient().format("${:,.2f}")

In [ ]:
%%kql rs << -ecbp
let _location = location;
let latestBill = toscalar(Usage | summarize max(BillingPeriodStartDate));
Usage
| where BillingPeriodStartDate == latestBill and ResourceLocation == _location
| {azmeta_kql_usage_scope}
| where ConsumedService == 'Microsoft.Compute' and MeterCategory == 'Virtual Machines' and isempty(ReservationId)
| summarize sum(Cost) by SubscriptionName, MeterSubCategory
| evaluate pivot(SubscriptionName, sum(sum_Cost))

In [ ]:
df: pd.DataFrame = rs.to_dataframe().set_index('MeterSubCategory')
df.style.background_gradient().format("${:,.2f}")

In [ ]:
%%kql rs << -ecbp
let latestBill = toscalar(Usage | summarize max(BillingPeriodStartDate));
Usage
| where BillingPeriodStartDate == latestBill
| {azmeta_kql_usage_scope}
| where ConsumedService == 'Microsoft.Compute' and MeterCategory == 'Virtual Machines' and isempty(ReservationId)
| summarize Cost=sum(Cost) by ResourceLocation, SubscriptionName, MeterSubCategory
| top 20 by Cost

In [ ]:
df: pd.DataFrame = rs.to_dataframe()
df.style.background_gradient(subset='Cost').format("${:,.2f}", subset='Cost')